<a href="https://colab.research.google.com/github/pcamarillor/O2024_ESI3914O/blob/Lab10_Team6/Copia_de_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,611 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,456 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,397

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, FloatType, StringType, IntegerType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

# Create Spark Session in localhost
spark = SparkSession.builder.master("local").\
    appName("ITESO-LogisticRegression").\
    getOrCreate()

# Set spark context
sc = spark.sparkContext
sc.setLogLevel("ERROR")

data_path = "/my_drive_path/Iris.csv"  # Cambia la ruta si es necesario
iris_df = spark.read.csv(data_path, header=True, inferSchema=True)
iris_df.show(5)

root
 |-- Id: integer (nullable = true)
 |-- SepalLengthCm: float (nullable = true)
 |-- SepalWidthCm: float (nullable = true)
 |-- PetalLengthCm: float (nullable = true)
 |-- PetalWidthCm: float (nullable = true)
 |-- Species: string (nullable = true)

+---+-------------+------------+-------------+------------+-----------+
|Id |SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|Species    |
+---+-------------+------------+-------------+------------+-----------+
|1  |5.1          |3.5         |1.4          |0.2         |Iris-setosa|
|2  |4.9          |3.0         |1.4          |0.2         |Iris-setosa|
|3  |4.7          |3.2         |1.3          |0.2         |Iris-setosa|
|4  |4.6          |3.1         |1.5          |0.2         |Iris-setosa|
|5  |5.0          |3.6         |1.4          |0.2         |Iris-setosa|
|6  |5.4          |3.9         |1.7          |0.4         |Iris-setosa|
|7  |4.6          |3.4         |1.4          |0.3         |Iris-setosa|
|8  |5.0          |3.4    

In [4]:
from pyspark.ml.feature import StringIndexer

label_indexer = StringIndexer(inputCol="Species", outputCol="label")
data = label_indexer.fit(iris_df).transform(iris_df)

data.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- SepalLengthCm: float (nullable = true)
 |-- SepalWidthCm: float (nullable = true)
 |-- PetalLengthCm: float (nullable = true)
 |-- PetalWidthCm: float (nullable = true)
 |-- Species: string (nullable = true)
 |-- label: double (nullable = false)



In [5]:
# Assemble the features into a single vector column
columns = ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]
assembler = VectorAssembler(inputCols=columns, outputCol="features")

data_with_features = assembler.transform(data).select("label", "features")

# Split the data: 80% training data and 20% testing data
train, test = data_with_features.randomSplit([0.8, 0.2], seed=13)

In [6]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

In [7]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

dt_model = dt.fit(train)

predictions = dt_model.transform(test)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})
print(f"Accuracy: {accuracy}")
precision = evaluator.evaluate(predictions,
{evaluator.metricName: "weightedPrecision"})
print(f"Precision: {precision}")
recall = evaluator.evaluate(predictions,
{evaluator.metricName: "weightedRecall"})
print(f"Recall: {recall}")
f1 = evaluator.evaluate(predictions,
{evaluator.metricName: "f1"})
print(f"F1 Score: {f1}")


Accuracy: 0.9629629629629629
Precision: 0.9675925925925926
Recall: 0.9629629629629629
F1 Score: 0.9632228719948018


In [8]:
from pyspark.ml.classification import DecisionTreeClassifier

dt_model = dt.fit(train)
# Display model summary
print("Decision Tree model summary:{0}".\
format(dt_model.toDebugString))


Decision Tree model summary:DecisionTreeClassificationModel: uid=DecisionTreeClassifier_a494a12d5c65, depth=5, numNodes=15, numClasses=3, numFeatures=4
  If (feature 2 <= 2.449999988079071)
   Predict: 0.0
  Else (feature 2 > 2.449999988079071)
   If (feature 3 <= 1.75)
    If (feature 2 <= 5.1499998569488525)
     If (feature 3 <= 1.6500000357627869)
      Predict: 1.0
     Else (feature 3 > 1.6500000357627869)
      If (feature 0 <= 4.950000047683716)
       Predict: 2.0
      Else (feature 0 > 4.950000047683716)
       Predict: 1.0
    Else (feature 2 > 5.1499998569488525)
     Predict: 2.0
   Else (feature 3 > 1.75)
    If (feature 2 <= 4.8500001430511475)
     If (feature 0 <= 5.950000047683716)
      Predict: 1.0
     Else (feature 0 > 5.950000047683716)
      Predict: 2.0
    Else (feature 2 > 4.8500001430511475)
     Predict: 2.0



In [10]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
f1 = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})
print(f'Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}')

Accuracy: 0.9629629629629629, Precision: 0.9675925925925926, Recall: 0.9629629629629629, F1: 0.9632228719948018
